<a href="https://colab.research.google.com/github/SandeeeeeeeeepDey/data-science-11-weeks-progg/blob/main/california_housing_wide_deep_keras_submodule_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To avoid static graph type usage in keras
> We use this dynamic graph structure to include
> *   Looping
*   Verity of shapes
*   Conditional branching



#Subclassing API in keras

##Pre

In [1]:
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
import pandas as pd

In [2]:
fetch = fetch_california_housing()

In [6]:
data = pd.DataFrame(fetch["data"], columns=fetch["feature_names"])
target = pd.Series(fetch["target"], name = fetch["target_names"][0])

In [7]:
train_size = int(len(data)*0.8)
test_size = round(len(data)*0.1)

In [20]:
X_train, y_train = data[:train_size], target[:train_size]
X_valid, y_valid = data[train_size: train_size+test_size], target[train_size: train_size+test_size]
X_test, y_valid = data[train_size+test_size:], target[train_size+test_size:]

In [21]:
X_train = X_train.iloc[:, :5], X_train.iloc[:, 2:]
X_valid = X_valid.iloc[:, :5], X_valid.iloc[:, 2:]
X_test = X_test.iloc[:, :5], X_test.iloc[:, 2:]

##Model

In [37]:
class WnD(tf.keras.Model):
  def __init__(self, units = 30, activation = "relu", **kwargs):
    super().__init__(**kwargs)
    self.norm_layer_wide = tf.keras.layers.Normalization()
    self.norm_layer_deep = tf.keras.layers.Normalization()
    self.hidden1 = tf.keras.layers.Dense(units, activation = activation)
    self.hidden2 = tf.keras.layers.Dense(units, activation = activation)
    self.main_output = tf.keras.layers.Dense(1)
    self.aux_output = tf.keras.layers.Dense(1)
  def call(self, inputs):
    input_wide, input_deep = inputs
    norm_wide = self.norm_layer_wide(input_wide)
    norm_deep = self.norm_layer_deep(input_deep)
    hidden1 = self.hidden1(norm_deep)
    hidden2 = self.hidden2(hidden1)
    concat = tf.keras.layers.concatenate([norm_wide, hidden2])
    output = self.main_output(concat)
    aux_output = self.aux_output(hidden2)
    return output, aux_output

model = WnD(name = "subclass_api")

In [38]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-3)

In [39]:
model.compile(loss="mse", optimizer = optimizer, metrics = "RootMeanSquaredError")

In [40]:
wide_t, deep_t = X_train
model.norm_layer_wide.adapt(wide_t)
model.norm_layer_deep.adapt(deep_t)

In [41]:
history = model.fit(X_train, y_train, epochs = 20, validation_data = (X_valid, y_valid))

Epoch 1/20
516/516 [==============================] - 4s 4ms/step - loss: 2.6940 - output_1_loss: 1.3477 - output_2_loss: 1.3463 - output_1_root_mean_squared_error: 1.1609 - output_2_root_mean_squared_error: 1.1603 - val_loss: 4.6870 - val_output_1_loss: 2.6327 - val_output_2_loss: 2.0544 - val_output_1_root_mean_squared_error: 1.6226 - val_output_2_root_mean_squared_error: 1.4333
Epoch 2/20
516/516 [==============================] - 2s 3ms/step - loss: 1.0844 - output_1_loss: 0.5026 - output_2_loss: 0.5819 - output_1_root_mean_squared_error: 0.7089 - output_2_root_mean_squared_error: 0.7628 - val_loss: 6.4172 - val_output_1_loss: 3.2972 - val_output_2_loss: 3.1200 - val_output_1_root_mean_squared_error: 1.8158 - val_output_2_root_mean_squared_error: 1.7664
Epoch 3/20
516/516 [==============================] - 2s 4ms/step - loss: 0.9465 - output_1_loss: 0.4269 - output_2_loss: 0.5195 - output_1_root_mean_squared_error: 0.6534 - output_2_root_mean_squared_error: 0.7208 - val_loss: 6.664

In [45]:
l, mse_main, rmse_main, mse_aux, mse_aux = model.evaluate(X_valid, y_valid)

65/65 [==============================] - 0s 3ms/step - loss: 4.6848 - output_1_loss: 2.4106 - output_2_loss: 2.2742 - output_1_root_mean_squared_error: 1.5526 - output_2_root_mean_squared_error: 1.5080


##Predict, Save and Use

In [52]:
y_pred_main, y_pred_aux = model.predict((X_test))
print(y_pred_main,"\n", y_pred_aux)

65/65 [==============================] - 0s 2ms/step
[[1.5743339 ]
 [1.8083241 ]
 [1.5315026 ]
 ...
 [0.7220373 ]
 [0.80612284]
 [0.78847456]] [[1.4250487 ]
 [2.2338011 ]
 [1.5747901 ]
 ...
 [0.9812301 ]
 [1.0145442 ]
 [0.80554473]]


Save

In [49]:
model.save("DnW", save_format="tf")

Reuse

In [53]:
model_in = tf.keras.models.load_model("DnW")
y_pred_main, y_pred_aux = model_in.predict(X_test)
print(y_pred_main,"\n", y_pred_aux)

65/65 [==============================] - 0s 2ms/step
[[1.5743339 ]
 [1.8083241 ]
 [1.5315025 ]
 ...
 [0.7220374 ]
 [0.80612284]
 [0.78847456]] 
 [[1.4250487 ]
 [2.2338011 ]
 [1.5747901 ]
 ...
 [0.9812301 ]
 [1.0145442 ]
 [0.80554473]]
